## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

### Data Load

In [3]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_20.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_20.csv')

DI_train = pd.read_csv('../data/DI_train_dataset_20.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_20.csv')

In [4]:
# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']

DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

### 인코딩 

In [5]:
IVF_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "임신_시도_또는_마지막_임신_경과_연수",
    "배란_유도_유형",
    "배아_생성_주요_이유",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이",
    "변환된_특정_시술_유형",
    "채취_해동_차이",
    "해동_혼합_차이",
    "혼합_이식_차이",
    "이식_해동_차이"
]

In [6]:
DI_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "임신_시도_또는_마지막_임신_경과_연수",
    "정자_기증자_나이",
    "변환된_특정_시술_유형"
]

In [7]:
# 모든 범주형 변수를 문자열로 변환
IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

In [8]:
# 데이터 분할
IVF_X_train, IVF_X_test, IVF_y_train, IVF_y_test = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

## Modeling

train 데이터의 일부를 검증데이터로 사용하여 성능지표 출력

In [9]:
# K-Fold를 사용하여 IVF와 DI의 잘못된 예측 데이터 수집
kf = KFold(n_splits=5, shuffle=True, random_state=42)

IVF_wrong_predictions = []
DI_wrong_predictions = []

# K-Fold 교차 검증을 통한 잘못된 예측 수집 함수
def collect_wrong_predictions(X, y, wrong_predictions, model_params):
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = lgb.LGBMClassifier(**model_params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        
        wrong_idx = val_idx[y_pred != y_val.values]
        wrong_predictions.append(X.iloc[wrong_idx])

# IVF와 DI에 대해 잘못된 예측 수집
IVF_model_params = {
    'n_estimators': 4471,
    'num_leaves': 13,
    'max_depth': 279,
    'learning_rate': 0.007075124517450591,
    'min_child_samples': 26,
    'subsample': 0.29772991936701476,
    'colsample_bytree': 0.8913054521763838,
    'reg_alpha': 0.0004860363321690653,
    'reg_lambda': 311.08056657247363,
    'min_split_gain': 0.18214905183450955,
    'random_state': 42,
    'boosting_type': 'gbdt',
    'verbose': -1
}

DI_model_params = {
    'n_estimators': 1816,
    'num_leaves': 3926,
    'max_depth': 259,
    'learning_rate': 0.00238377640011148,
    'min_child_samples': 1,
    'subsample': 0.7610056627240331,
    'colsample_bytree': 0.6655579164853634,
    'reg_alpha': 0.00025227758337188327,
    'reg_lambda': 76.744107215122684,
    'min_split_gain': 0.007773520329665474,
    'random_state': 42,
    'boosting_type': 'gbdt',
    'verbose': -1
}

collect_wrong_predictions(IVF_X, IVF_y, IVF_wrong_predictions, IVF_model_params)
collect_wrong_predictions(DI_X, DI_y, DI_wrong_predictions, DI_model_params)

In [62]:
# 잘못된 예측 데이터 결합
IVF_wrong_data = pd.concat(IVF_wrong_predictions).fillna(0)
DI_wrong_data = pd.concat(DI_wrong_predictions).fillna(0)

# KMeans 군집화 모델 학습 (IVF)
kmeans_IVF = KMeans(n_clusters=2, random_state=42)
clusters_IVF = kmeans_IVF.fit_predict(IVF_wrong_data)
IVF_wrong_data['cluster'] = clusters_IVF

# KMeans 군집화 모델 학습 (DI)
kmeans_DI = KMeans(n_clusters=2, random_state=42)
clusters_DI = kmeans_DI.fit_predict(DI_wrong_data)
DI_wrong_data['cluster'] = clusters_DI

# 군집화된 데이터 확인
print("IVF 군집화 결과:")
print(IVF_wrong_data.groupby('cluster').size())
print("DI 군집화 결과:")
print(DI_wrong_data.groupby('cluster').size())

# 전체 train 데이터에 대해 LGBM 모델 학습 (IVF)
IVF_model_final = lgb.LGBMClassifier(**IVF_model_params)
IVF_model_final.fit(IVF_X, IVF_y)

# 전체 train 데이터에 대해 LGBM 모델 학습 (DI)
DI_model_final = lgb.LGBMClassifier(**DI_model_params)
DI_model_final.fit(DI_X, DI_y)

# 전체 train 데이터에 대해 랜덤 포레스트 모델 학습 (IVF)
IVF_rf_model = RandomForestClassifier(random_state=42)
IVF_rf_model.fit(IVF_X, IVF_y)

# 전체 train 데이터에 대해 랜덤 포레스트 모델 학습 (DI)
DI_rf_model = RandomForestClassifier(random_state=42)
DI_rf_model.fit(DI_X, DI_y)

# 테스트 데이터에 대한 확률 예측 수행 (IVF)
IVF_test_pred_proba = IVF_model_final.predict_proba(IVF_test.drop('ID', axis=1))[:, 1]

# 테스트 데이터에 대한 확률 예측 수행 (DI)
DI_test_pred_proba = DI_model_final.predict_proba(DI_test.drop('ID', axis=1))[:, 1]

# 테스트 데이터에 대한 군집화 예측 (IVF)
IVF_test_data = IVF_test.drop(['ID'], axis=1).fillna(0)
test_clusters_IVF = kmeans_IVF.predict(IVF_test_data)

# 테스트 데이터에 대한 군집화 예측 (DI)
DI_test_data = DI_test.drop(['ID'], axis=1).fillna(0)
test_clusters_DI = kmeans_DI.predict(DI_test_data)

# 군집화 모델이 잘못된 예측을 할 것으로 예측한 행의 확률값을 랜덤 포레스트로 예측 (IVF)
IVF_test_pred_proba[test_clusters_IVF == 1] = IVF_rf_model.predict_proba(IVF_test_data[test_clusters_IVF == 1])[:, 1]

# 군집화 모델이 잘못된 예측을 할 것으로 예측한 행의 확률값을 랜덤 포레스트로 예측 (DI)
DI_test_pred_proba[test_clusters_DI == 1] = DI_rf_model.predict_proba(DI_test_data[test_clusters_DI == 1])[:, 1]

# 최종 제출 파일 생성
IVF_submission = pd.DataFrame({'ID': IVF_test['ID'], 'probability': IVF_test_pred_proba})
DI_submission = pd.DataFrame({'ID': DI_test['ID'], 'probability': DI_test_pred_proba})

submission = pd.concat([IVF_submission, DI_submission])
submission.to_csv('../submission/code21_1_submit.csv', index=False)

print("제출 파일(code21_1_submit.csv) 생성 완료!")

IVF 군집화 결과:
cluster
0    50708
1    13532
dtype: int64
DI 군집화 결과:
cluster
0    487
1    324
dtype: int64
제출 파일(code21_1_submit.csv) 생성 완료!


데이콘 PUBLIC 0.7301773912

엑셀 제대로 뽑혔나 확인

In [63]:
import pandas as pd

# 두 CSV 파일을 읽어옵니다.
df1 = pd.read_csv('../submission/code21_1_submit.csv')
df2 = pd.read_csv('../submission/code21_submit.csv')

# '임신_성공_여부' 열을 비교합니다.
comparison = df1['probability'] == df2['probability']

# 차이점이 있는 행을 출력합니다.
differences = df1[~comparison]

len(differences)

86174

In [64]:
import pandas as pd

# 두 CSV 파일을 읽어옵니다.
df1 = pd.read_csv('../submission/code21_1_submit.csv')
df2 = pd.read_csv('../submission/code20_submit.csv')

# '임신_성공_여부' 열을 비교합니다.
comparison = df1['probability'] == df2['probability']

# 차이점이 있는 행을 출력합니다.
differences = df1[~comparison]

len(differences)

90048

In [65]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90067 entries, 0 to 90066
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           90067 non-null  object 
 1   probability  90067 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


In [68]:
df1.tail(10)

,ID,probability
90057,TEST_89734,0.086779
90058,TEST_89759,0.057310
90059,TEST_89786,0.320000
90060,TEST_89846,0.101747
90061,TEST_89865,0.010000
90062,TEST_89872,0.203833
90063,TEST_89907,0.180000
90064,TEST_89940,0.060899
90065,TEST_90001,0.164117
90066,TEST_90046,0.094943


In [69]:
df2.tail(10)

,ID,probability
90057,TEST_90057,0.311083
90058,TEST_90058,0.213900
90059,TEST_90059,0.419320
90060,TEST_90060,0.273916
90061,TEST_90061,0.401884
90062,TEST_90062,0.001331
90063,TEST_90063,0.300819
90064,TEST_90064,0.520521
90065,TEST_90065,0.216879
90066,TEST_90066,0.001120


In [32]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90067 entries, 0 to 90066
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           90067 non-null  object 
 1   probability  90067 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


In [26]:
df3 = pd.read_csv('../submission/sample_submission.csv')

In [27]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90067 entries, 0 to 90066
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           90067 non-null  object 
 1   probability  90067 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


.